Antoni Pawłowicz 191733

<div align="center">
<em>
METODY INTERAKCJI CZŁOWIEK MASZYNA</br></br>
Laboratoria 2</br>
Algorytmy estymacji i detekcji postawy ciała
</em>

In [1]:
import cv2
import numpy as np
import copy

### _Zadanie 1_
Używając wcześniej zdefiniowanych par połącz odpowiednie punkty liniami
tworząc szkielet obiektu.

In [2]:
protoTxt = "OpenPose/pose_deploy_linevec_faster_4_stages.prototxt"
wagi = "OpenPose/pose_iter_160000.caffemodel"
cam_id = '/home/anthonyp57/VSCode_projects/MICM_2025/lab_3/man_vid.mp4'

net = cv2.dnn.readNetFromCaffe(protoTxt, wagi)

img = cv2.imread(cam_id)

# here for clarity
part_to_idx = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
"LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
"RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
"Background": 15}

BODY_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder",
"RElbow"],["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder",
"LElbow"],["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"],
["RHip", "RKnee"],["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip",
"LKnee"], ["LKnee", "LAnkle"] ]

pairs_idx = [(part_to_idx[a[0]], part_to_idx[a[1]]) for a in BODY_PAIRS]
part_indices = part_to_idx.values()[:-1] #no backgound
part_names = part_to_idx.keys()[:-1]
threshold = 0.6

limb_pos = {i:None for i in part_indices}

def lines_to_limbs(image):
    img = copy.deepcopy(image)
    inWidth = img.shape[1]
    inHeight = img.shape[0]

    inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (inWidth,
    inHeight), (0, 0, 0), swapRB=False, crop=False)

    # ustawienie jako input
    net.setInput(inpBlob)
    #predykcja
    output = net.forward()

    limb_map = output[0,:len(part_indices),:,:]

    for i,l in enumerate(limb_map):

        probMap = cv2.resize(l, (inWidth, inHeight))
        # zamiana skali kolorów dla confidence map i oznaczenie kolorem czerwonym
        probMap2 = cv2.cvtColor(probMap, cv2.COLOR_GRAY2RGB) #part prob
        probMap2[:,:,0]=0
        probMap2[:,:,1]=0

        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.addWeighted(img, 0.4, probMap2, 0.6, 0, dtype=cv2.CV_32F).astype(np.uint8)

        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            x,y = point
            x,y = int(x),int(y)
            cv2.circle(img, (x,y), 5, (255, 255, 255), thickness=2, lineType=cv2.FILLED)
            cv2.putText(img, f'{part_names[i]}', (x-5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3, lineType=cv2.LINE_AA)
            limb_pos[i] = (x,y)
        else :
            limb_pos[i] = None

    #draw lines between connecting body parts
    for p in pairs_idx:
        cv2.line(img, limb_pos[p[0], limb_pos[p[1]]], 3)

    cv2.imshow('posenet', img)

lines_to_limbs(img)

error: OpenCV(4.3.0) /tmp/pip-req-build-n_rdqh6s/opencv/modules/dnn/src/caffe/caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "OpenPose/pose_deploy_linevec_faster_4_stages.prototxt" in function 'ReadProtoFromTextFile'


### _Zadanie 2_
 Stwórz algorytm detekcji dowolnej pozy ciała na podstawie wzajemnego
położenia punktów kluczowych (ich wzajemnych odległości, kątów pomiędzy
połączeniami par itp.). Dokładnie opisz działanie algorytmu i przedstaw wyniki jego
działania.

In [ ]:
def calculate_angle_between_points(x, y, z):
    v1 = y - x
    v2 = z - y
    
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle = np.arccos(cos_theta)
    angle = np.degrees(angle)

    return angle

def get_limb(prob):

    minVal, prob, minLoc, point = cv2.minMaxLoc(prob)

    if prob > threshold :
        x,y = point
        x,y = int(x),int(y)
    
    return x,y

def is_cactus(image):
    img = copy.deepcopy(image)
    inWidth = img.shape[1]
    inHeight = img.shape[0]

    inpBlob = cv2.dnn.blobFromImage(img, 1.0 / 255, (inWidth,
    inHeight), (0, 0, 0), swapRB=False, crop=False)

    # ustawienie jako input
    net.setInput(inpBlob)
    #predykcja
    output = net.forward()

    limb_map = output[0,:len(part_indices),:,:]

    neck = get_limb(limb_map[part_to_idx['Neck']])
    lwrist = get_limb(limb_map[part_to_idx['LWrist']])
    rwrist = get_limb(limb_map[part_to_idx['RWrist']])
    lelbow = get_limb(limb_map[part_to_idx['LElbow']])
    relbow = get_limb(limb_map[part_to_idx['RElbow']])
    lshoulder = get_limb(limb_map[part_to_idx['LShoulder']])
    rshoulder = get_limb(limb_map[part_to_idx['RShoulder']])

    rshoulder_elbow_wrist_angle = calculate_angle_between_points(rshoulder, relbow, rwrist)
    lshoulder_elbow_wrist_angle = calculate_angle_between_points(lshoulder, lelbow, lwrist)

    


    cv2.imshow('posenet', img)


90.0
